<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset el libro "The Adventures of Sherlock Holmes" de Arthur Conan Doyle

In [103]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('the_adventures_of_sherlock_holmes.txt', sep='/n', engine='python', header=None)
df.head()

,0
0,The Project Gutenberg eBook of The Adventures ...
1,This eBook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


In [104]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9626


### 1 - Preprocesamiento

In [105]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [107]:
# Demos un vistazo
sentence_tokens[50:52]

[['adler', 'of', 'dubious', 'and', 'questionable', 'memory'],
 ['i',
  'had',
  'seen',
  'little',
  'of',
  'holmes',
  'lately',
  'my',
  'marriage',
  'had',
  'drifted',
  'us',
  'away']]

### 2 - Crear los vectores (word2vec)

In [108]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [109]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                    window=2,       # cant de palabras antes y desp de la predicha
                    vector_size=400,       # dimensionalidad de los vectores 
                    negative=20,    # cantidad de negative samples... 0 es no se usa
                    workers=1,      # si tienen más cores pueden cambiar este valor
                    sg=1)           # modelo 0:CBOW  1:skipgram

In [110]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [111]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9626


In [112]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index))

Cantidad de words distintas en el corpus: 2231


### 3 - Entrenar el modelo generador

In [113]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=20,
                compute_loss = True,
                callbacks=[callback()]
                )

Loss after epoch 0: 882543.625
Loss after epoch 1: 613979.25
Loss after epoch 2: 582623.625
Loss after epoch 3: 503463.75
Loss after epoch 4: 498194.0
Loss after epoch 5: 496698.0
Loss after epoch 6: 491409.25
Loss after epoch 7: 469579.5
Loss after epoch 8: 458565.5
Loss after epoch 9: 454541.0
Loss after epoch 10: 452995.0
Loss after epoch 11: 448325.0
Loss after epoch 12: 446506.0
Loss after epoch 13: 444282.5
Loss after epoch 14: 440983.0
Loss after epoch 15: 438874.0
Loss after epoch 16: 431306.0
Loss after epoch 17: 420712.0
Loss after epoch 18: 419144.0
Loss after epoch 19: 421645.0


(1358642, 2224020)

### 4 - Ensayar

In [114]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["watson"], topn=10)

[('doctor', 0.833378255367279),
 ('suppose', 0.7919176816940308),
 ('“ha', 0.7917171120643616),
 ('prison', 0.7883281111717224),
 ('scoundrel', 0.7791693210601807),
 ('“certainly', 0.7760857939720154),
 ('patient', 0.7757074236869812),
 ('begin', 0.7752490043640137),
 ('madam', 0.7737632393836975),
 ('“ah', 0.7714954018592834)]

In [117]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["police"], topn=10)

[('facts', 0.7778236865997314),
 ('county', 0.7691174149513245),
 ('examine', 0.7650518417358398),
 ('goodness', 0.7591449022293091),
 ('ball', 0.758080244064331),
 ('cabman', 0.7580599784851074),
 ('deed', 0.7526472210884094),
 ('murderer', 0.7503688335418701),
 ('europe', 0.7496589422225952),
 ('paddington', 0.7493327856063843)]

In [121]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sherlock"], topn=10)

[('lestrade', 0.7491377592086792),
 ('engineer', 0.7396234273910522),
 ('ross', 0.7299661636352539),
 ('“mr', 0.7264847159385681),
 ('laughing', 0.7201484441757202),
 ('jabez', 0.717681884765625),
 ('adventures', 0.7174314856529236),
 ('duncan', 0.715665876865387),
 ('“come', 0.7127528190612793),
 ('gravely', 0.7126884460449219)]

In [122]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('keeping', 0.8034171462059021),
 ('commission', 0.7993332743644714),
 ('service', 0.7904911637306213),
 ('poison', 0.7870985865592957),
 ('metallic', 0.7744030356407166)]

### 5 - Visualizar agrupación de vectores

In [123]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [124]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer='vscode') # esto para plotly en colab

c:\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

c:\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### Alumno

- Se observaron algunos resultados interesantes al buscar palabras relacionadas:
* Watson -> doctor, certainly
* Police -> facts, county, examine, murderer
* Money -> service, comission, poison
* Sherlock -> Lestrade, engineer, adventure
- Por otro lado, analizando el gráfico de los embeddings en 2D no están claras las relaciones o similitudes entre palabras. Posiblemente con mayor analisis exploratorio del corpus y mejorando el preprocesamiento se puedan mejorar los resultados.